<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Transformer_Practicse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Chapter 2 Text Classification

In [2]:
# Transformer practicse

In [3]:
!pip install datasets
!pip install transformers
!pip3 install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 9.4 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
emotions = load_dataset("emotion")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [6]:
emotions['train']['text'][0]

'i didnt feel humiliated'

In [7]:
emotions['train']['label'][0]

0

In [8]:
check = load_dataset('csv',data_files = "/content/checkcsv.csv")

FileNotFoundError: ignored

In [ ]:
check

In [ ]:
check.set_format(type="pandas")

In [ ]:
check['train'][:]

In [9]:
from transformers import AutoTokenizer

model_chpt = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_chpt)

In [10]:
encoded_text= tokenizer(['This is a greate movie'])

In [11]:
tokenizer.vocab_size

30522

In [12]:
encoded_text

{'input_ids': [[101, 2023, 2003, 1037, 2307, 2063, 3185, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}

In [13]:
def tokenize(batch):
  return tokenizer(batch['text'],padding= True , truncation =True)

In [22]:
import torch
from transformers import AutoModelForSequenceClassification
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
num_labels = 6
model =AutoModelForSequenceClassification.from_pretrained(model_chpt).to(device)

In [23]:
emotions_encoded =emotions.map(tokenize,batched=True,batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [24]:
emotions_encoded['train'].column_names

['text', 'label', 'input_ids', 'attention_mask']

In [25]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [28]:



from transformers import Trainer, TrainingArguments
batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_chpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                      num_train_epochs=2,
                                      learning_rate=2e-5,
                                      per_device_train_batch_size=batch_size,
                                      per_device_eval_batch_size=batch_size,
                                      weight_decay=0.01,
                                      evaluation_strategy="epoch",
                                      disable_tqdm=False,
                                      logging_steps=logging_steps,
                                   #   push_to_hub=True,
                                      log_level="error")

trainer = Trainer(model=model,
                  args=training_args,
                    compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)


RuntimeError: ignored

In [20]:

trainer.train()

'cuda'